# Trabajo PLN : Clasificación Multietiqueta


Alumnos: Jorge Albalat, Andreu Cantó, Amparo Gálvez y Mario Herranz

## 0. Carga de todas las librerías

In [1]:
import spacy
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score, precision_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV

## 1. Pre-Procesado

En primer lugar, cargamos los datos de entrenamiento y prueba. Los datos consisten en tweets etiquetados con múltiples emociones. Utilizamos el conjunto de entrenamiento para entrenar los modelos y el conjunto de prueba para evaluar su rendimiento.

In [2]:
# Cargar el modelo de lenguaje en español
nlp = spacy.load("es_core_news_sm")

# Añadir el pipe para merge_entities
nlp.add_pipe("merge_entities")

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

### Cargamos los datos

In [3]:
# Cargar conjunto de entrenamiento
df = pd.read_csv('sem_eval_train_es.csv')

# Visualizar las primeras filas para verificar que se hayan cargado correctamente
print("Conjunto de entrenamiento:")
print(df.head())

Conjunto de entrenamiento:
              ID                                              Tweet  anger  \
0  2018-Es-01643  @aliciaenp Ajajjaa somos del clan twitteras pe...  False   
1  2018-Es-05142  @AwadaNai la mala suerte del gato fichame la c...  False   
2  2018-Es-05379  @audiomano A mí tampoco me agrado mucho eso. E...   True   
3  2018-Es-00208  Para llevar a los bebes de un lugar a otro deb...  False   
4  2018-Es-01385  @DalasReview me encanta la terrible hipocresia...   True   

   anticipation  disgust   fear    joy   love  optimism  pessimism  sadness  \
0         False    False  False   True  False     False      False    False   
1         False    False   True  False  False     False       True    False   
2         False    False  False  False  False     False      False    False   
3         False    False  False   True  False     False      False    False   
4         False     True  False  False  False     False      False    False   

   surprise  trust  
0     Fa

## 2. Limpieza y normalización de texto

Para preparar los datos textuales para su uso en modelos de aprendizaje automático, aplicamos una serie de técnicas de limpieza y normalización, incluyendo la eliminación de caracteres especiales, la conversión a minúsculas y la normalización de las palabras.

In [4]:
# Función para limpiar el texto
def clean_text(text):
    # Eliminar menciones y URL
    text = re.sub(r'@[A-Za-z0-9_]+|https?://[^ ]+', '', text)

    # Eliminar el carácter '#' de los hashtags
    text = re.sub(r'#', '', text)

    # Eliminar signos de puntuación y palabras menores de 3 caracteres
    text = re.sub(r'\b\w{1,2}\b|[^a-zA-Z\s]', '', text)

    # Lematización del texto
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    # Eliminar emoticonos y caracteres especiales
    lemmatized_text = re.sub(r'[^\w\s]', '', lemmatized_text)

    return lemmatized_text

In [5]:
# Limpiar el texto
df['Clean_Text'] = df['Tweet'].apply(clean_text)
df['Clean_Text'].head()

0      Ajajjaa ser del clar twittera perdido   even...
1       malo suerte del gato fichame   cara   help ...
2        tampoco   agrado mucho ese especialmente p...
3    para llevar   el bebes    lugar   otro deber c...
4       encanta   terrible hipocresia   doble moral...
Name: Clean_Text, dtype: object

### Tokenizado de los datos

In [6]:
# Función que normaliza un conjunto de tweets limpios
def normaliza(texto):
    """Función que normaliza un string de texto
    Entrada: string a normalizar
    Devuelve: string del texto normalizado"""
    doc = nlp(texto)
    norm = []
    for token in doc:
        if not token.is_stop and not token.is_punct:
            lema = token.lemma_
            # Si el token es una entidad nombrada, lo reemplazamos con una etiqueta específica
            if token.ent_type_ == 'PER':
                lema = 'PERSONA'
            elif token.ent_type_ == 'LOC':
                lema = 'LUGAR'
            elif token.ent_type_ == 'ORG':
                lema = 'ORGANIZACIÓN'
            elif token.ent_type_ == 'MISC':
                lema = 'OTRO'

            norm.append(lema)

    return ' '.join(norm)

In [7]:
# Normalizar el texto (opcional)
df['Normalized_Text'] = df['Clean_Text'].apply(normaliza)
df['Normalized_Text'].head()

0       PERSONA clar twittera perdido    evento imp...
1        malo suerte gato fichame    cara    helpir...
2            agrado especialmente tratar      justi...
3       bebes     lugar    deber cantarl canción   ...
4        encantar    terrible hipocresia    doble m...
Name: Normalized_Text, dtype: object

## 3. Vectorización del texto

Convertimos el texto normalizado en vectores de características utilizando la matriz BoW.

In [8]:
# Crear la matriz BoW usando Normalized_Text
vectorizador = CountVectorizer()
X = vectorizador.fit_transform(df['Normalized_Text']).toarray()

# Convertir las etiquetas a formato binario
y = df[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].values

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Modelado y entrenamiento

### Red Neuronal Multicapa (MLP)

In [9]:
# Definición del modelo secuencial
model_mlp = Sequential()

# Capa densa con 128 unidades y activación ReLU, con entrada del tamaño de X_train
model_mlp.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# Capa densa con 64 unidades y activación ReLU
model_mlp.add(Dense(64, activation='relu'))

# Capa de salida con 11 unidades (correspondientes a 11 etiquetas) y activación sigmoide para salida binaria
model_mlp.add(Dense(11, activation='sigmoid'))

# Compilación del modelo con optimizador Adam y pérdida de entropía cruzada binaria
model_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Definición de callbacks: guardado del mejor modelo y parada temprana
checkpoint = ModelCheckpoint('best_model_mlp.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenamiento del modelo con validación del 20% de los datos, durante 50 épocas y con tamaño de lote de 32
history_mlp = model_mlp.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[checkpoint, early_stopping])

Epoch 1/50
63/72 [=========================>....] - ETA: 0s - loss: 0.5406 - accuracy: 0.1562
Epoch 1: val_loss improved from inf to 0.40788, saving model to best_model_mlp.h5
72/72 [==============================] - 1s 9ms/step - loss: 0.5238 - accuracy: 0.1874 - val_loss: 0.4079 - val_accuracy: 0.3509
Epoch 2/50
17/72 [======>.......................] - ETA: 0s - loss: 0.3816 - accuracy: 0.4375

C:\Users\ampar\Anaconda3\envs\my_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


71/72 [============================>.] - ETA: 0s - loss: 0.3743 - accuracy: 0.4912
Epoch 2: val_loss improved from 0.40788 to 0.38713, saving model to best_model_mlp.h5
72/72 [==============================] - 0s 7ms/step - loss: 0.3745 - accuracy: 0.4917 - val_loss: 0.3871 - val_accuracy: 0.4316
Epoch 3/50
65/72 [==========================>...] - ETA: 0s - loss: 0.3152 - accuracy: 0.5529
Epoch 3: val_loss improved from 0.38713 to 0.36663, saving model to best_model_mlp.h5
72/72 [==============================] - 0s 7ms/step - loss: 0.3131 - accuracy: 0.5562 - val_loss: 0.3666 - val_accuracy: 0.4333
Epoch 4/50
67/72 [==========================>...] - ETA: 0s - loss: 0.2309 - accuracy: 0.6614
Epoch 4: val_loss improved from 0.36663 to 0.35739, saving model to best_model_mlp.h5
72/72 [==============================] - 0s 7ms/step - loss: 0.2301 - accuracy: 0.6615 - val_loss: 0.3574 - val_accuracy: 0.4632
Epoch 5/50
68/72 [===========================>..] - ETA: 0s - loss: 0.1602 - accurac

### Regresión Logística Multietiqueta

In [10]:
# Parámetros de búsqueda en cuadrícula
param_grid = {
    'estimator__C': [0.01, 0.1, 1, 10, 100],
    'estimator__solver': ['newton-cg', 'lbfgs', 'liblinear']
}

# Definir el modelo de regresión logística multietiqueta
model_lr = MultiOutputClassifier(LogisticRegression())

# Configuración de búsqueda en cuadrícula
grid_search_lr = GridSearchCV(model_lr, param_grid, cv=3, scoring='accuracy')

# Ajustar el modelo
grid_search_lr.fit(X_train, y_train)

# Extraer los mejores parámetros
best_params = grid_search_lr.best_params_
print(f"Best Logistic Regression params: {best_params}")

# Crear un nuevo modelo LogisticRegression con los mejores parámetros
best_estimator_params = {
    'C': best_params['estimator__C'],
    'solver': best_params['estimator__solver']
}
best_lr_model = LogisticRegression(**best_estimator_params)

C:\Users\ampar\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ampar\Anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

Best Logistic Regression params: {'estimator__C': 10, 'estimator__solver': 'liblinear'}


In [11]:
# Crear el MultiOutputClassifier con el mejor modelo LogisticRegression
best_lr = MultiOutputClassifier(best_lr_model)

# Entrenar el modelo con los mejores parámetros
best_lr.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(C=10, solver='liblinear'))

### Random Forest

In [12]:
# Definición de la distribución de parámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': randint(10, 200),
    'max_depth': randint(1, 100),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'bootstrap': [True, False]
}

# Definición del modelo Random Forest
rf = RandomForestClassifier(n_jobs=-1)

# Configuración de la búsqueda aleatoria con 50 iteraciones y 3 folds de validación cruzada
random_search_rf = RandomizedSearchCV(rf, param_dist, n_iter=50, cv=3, scoring='accuracy', n_jobs=-1, random_state=42)

# Ajuste del modelo utilizando la búsqueda aleatoria
random_search_rf.fit(X_train, y_train)

# Extracción del mejor estimador (modelo) y sus parámetros óptimos
best_rf = random_search_rf.best_estimator_
print(f"Best Random Forest params: {random_search_rf.best_params_}")

Best Random Forest params: {'bootstrap': False, 'max_depth': 86, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 110}


In [13]:
rf = RandomForestClassifier(bootstrap=False,n_estimators=110, max_depth=86, min_samples_leaf=3, min_samples_split=2)
# Entrenamiento del modelo con los mejores parámetros
best_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=86, min_samples_leaf=3,
                       n_estimators=110, n_jobs=-1)

## 5. Evaluación de los Modelos

Evaluamos los modelos utilizando el conjunto de validación y comparamos sus métricas de rendimiento.

In [14]:
# Cargar y evaluar el modelo MLP
model_mlp.load_weights('best_model_mlp.h5')
loss_mlp, accuracy_mlp = model_mlp.evaluate(X_val, y_val)

# Cargar y evaluar el modelo Random Forest
loss_rf, accuracy_rf = best_rf.score(X_val, y_val), accuracy_score(y_val, best_rf.predict(X_val))

# Cargar y evaluar el modelo Regresion Logistica
loss_lr, accuracy_lr = best_lr.score(X_val, y_val), accuracy_score(y_val, best_lr.predict(X_val))

23/23 [==============================] - 0s 2ms/step - loss: 0.3552 - accuracy: 0.4502


In [15]:
# Almacenar los resultados en una lista
results = [
    {'model': 'MLP', 'loss': loss_mlp, 'accuracy': accuracy_mlp},
    {'model': 'Logistic Regression', 'loss': loss_lr, 'accuracy': accuracy_lr},
    {'model': 'Random Forest', 'loss': loss_rf, 'accuracy': accuracy_rf}
]

# Mostrar todos los resultados a la vez
for result in results:
    print(f"Modelo: {result['model']}")
    print(f"  Pérdida en validación: {result['loss']}")
    print(f"  Precisión en validación: {result['accuracy']}")

Modelo: MLP
  Pérdida en validación: 0.3552181124687195
  Precisión en validación: 0.4502103924751282
Modelo: Logistic Regression
  Pérdida en validación: 0.19074333800841514
  Precisión en validación: 0.19074333800841514
Modelo: Random Forest
  Pérdida en validación: 0.15287517531556802
  Precisión en validación: 0.15287517531556802


## 6. Predicción en el Conjunto de Test

Realizamos predicciones con el modelo MLP en el conjunto de prueba y guardamos los resultados.

In [16]:
# Predecir en el conjunto de test
df_test = pd.read_csv('sem_eval_test_grupo_12.csv')
df_test['Clean_Text'] = df_test['Tweet'].apply(clean_text)
df_test['Normalized_Text'] = df_test['Clean_Text'].apply(normaliza)
X_test = vectorizador.transform(df_test['Normalized_Text']).toarray()
predicciones = model_mlp.predict(X_test)
predicciones_binarias = (predicciones > 0.5).astype(int)

soluciones_df = pd.DataFrame(predicciones_binarias, columns=['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'])
soluciones_df.insert(0, 'ID', df_test['ID'])
soluciones_df.to_csv('soluciones_grupo_12.csv', index=False)

22/22 [==============================] - 0s 2ms/step


### Conclusión

En este trabajo, hemos desarrollado y evaluado diferentes modelos de clasificación multietiqueta para la detección de emociones en tweets en español. Utilizamos una red neuronal multicapa (MLP), regresión logística y un random forest, optimizando los hiperparámetros de cada modelo y comparando su rendimiento en términos de pérdida y precisión en el conjunto de validación.

La red neuronal multicapa (MLP) mostró una precisión superior en la clasificación de emociones en comparación con la regresión logística y el random forest, a pesar de tener una mayor pérdida. Esto sugiere que la MLP tiene una mejor capacidad para capturar las complejidades de los datos multietiqueta, aunque podría beneficiarse de una mayor optimización para reducir la pérdida.

Por otro lado, la regresión logística y el random forest presentaron una pérdida menor, pero su precisión fue inferior, lo que indica que aunque estos modelos son buenos para minimizar la función de pérdida, tienen dificultades para predecir correctamente las etiquetas emocionales en comparación con la MLP.

En conclusión, la MLP demostró ser el modelo más efectivo para la tarea de clasificación multietiqueta de emociones en tweets en español.